Llegir arxiu

In [ ]:
import numpy as np

# Función para leer el archivo y extraer las matrices
def leer_archivo(nombre_archivo):
    c = []
    A = []
    b = []

    with open(nombre_archivo, 'r') as file:
        for line in file:
            if 'c=' in line:
                while True:
                    line = next(file)
                    c.append([int(x) for x in line.split()])
                    if line=='\n':
                        c.pop()                      
                        break 
            elif 'A=' in line:
                while True:
                    line = next(file)                    
                    A.append([int(x) for x in line.split()])
                    if line=='\n':  
                        A.pop()                    
                        break 

            elif 'b=' in line:
                while True:
                    line = next(file)
                    b.append([int(x) for x in line.split()])
                    if line=='\n': 
                        b.pop()                     
                        break 
    # Convertir las listas a arrays numpy
    c = np.array(c)
    A = np.array(A)
    b = np.array(b)
    
    return c, A, b

# Nombre del archivo
nombre_archivo = 'tes.txt'

# Llamar a la función para leer el archivo
c, A, b = leer_archivo(nombre_archivo)

# Mostrar las matrices
print("Matriz c:")
print(c)
print("\nMatriz A:")
print(A)
print("\nVector b:")
print(b)

Classe

In [ ]:
class SIMPLEX:
    def __init__(self,Coeficientes, Arestricciones, bsolucion):
    
    def encontrar_SBF_fase1:
    
    def optima_variablesentrada:
    
    def DBF_:
    
    def longitudpaso_quiensale:
    
    def actualizar y repetir: